# EO-Forge Loaders 1.0.1 - Logging

This notebooks showcase the logging functionality built-in into eo-forge. 

By default, the library will try to use the root logger. If it is not defined, the library sets a default logger
that prints the information to the stderr (min level=DEBUG). The logger is "eo_forge.default" and it is located at
`eo_forge.default_logger`. Note that this logger is used in all the modules of the library by default.

Below we show a few usage examples of the logging capabilities.
To that end, we assume that you have already downloaded some images (we cover the download steps from GCP in other [notebook](./eo-forge-gcp-downloader.ipynb)):

- Landsat 5/8 - Path/Row : 230/094
- Sentinel2 - Tile: 19FCF

## Default logging

Here we will show how the default loggin looks like.

In [1]:
################################
# General imports and definitions

import os

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio as rio
from earthpy import plot

import eo_forge
from eo_forge.io.LandsatLoaders import LandsatLoader
from eo_forge.io.SentinelLoaders import Sentinel2Loader

# Base dir
TMP_PROD = "./products-int/"
# IMAGES RAW Search Path
IMAGES_RAW = os.path.join(TMP_PROD, "images-raw")
#
# IMAGES CROP Path (to leave cropped files - if appy)
PROD_INT_DIR = os.path.join(TMP_PROD, "images-crop")
os.makedirs(PROD_INT_DIR, exist_ok=True)

In [2]:
# Init Loader and query bands
LANDSAT5_BANDS = ["B1", "B2", "B3", "B4", "B5"]
lp = LandsatLoader(IMAGES_RAW, bands=LANDSAT5_BANDS, spacecraft=5)

2022/02/09 12:07:49 - INFO - Running on Landsat 5 data


That is, we have instantiated the Loader with:
- folder: IMAGE_RAW, the place where we look for products (see later)
- bands: list of bands that we require to be processed
- spacecraft: 5
- resolution: leave default @ 30 meters
- reflectance: leave default as True so calculate reflectance @ TOA (otherwise calculate radiance)

And now call the processor for the requested product-id:

In [3]:
product_id = "LT05_L1TP_230094_20111011_20161005_01_T1"
res_ = lp.execute(
    product_id,
    bbox=None,
    folder_proc_=PROD_INT_DIR,
    raster_return_open=False,
    write_file="_complete",
)

2022/02/09 12:07:49 - INFO - Processing on ./products-int/images-raw/LT05_L1TP_230094_20111011_20161005_01_T1 dir
2022/02/09 12:07:49 - INFO - Using clipping flag: False
2022/02/09 12:07:49 - INFO - PROCESSING band: B1
2022/02/09 12:07:50 - INFO - resample: False - scale factor 1 - true pixel 30.0
2022/02/09 12:07:50 - INFO - no bbox - full match: True - area: 1
2022/02/09 12:07:50 - INFO - calibrating band
2022/02/09 12:07:53 - INFO - reprojecting band
2022/02/09 12:08:02 - INFO - PROCESSING band: B2
2022/02/09 12:08:02 - INFO - resample: False - scale factor 1 - true pixel 30.0
2022/02/09 12:08:02 - INFO - no bbox - full match: True - area: 1
2022/02/09 12:08:03 - INFO - calibrating band
2022/02/09 12:08:05 - INFO - reprojecting band
2022/02/09 12:08:14 - INFO - PROCESSING band: B3
2022/02/09 12:08:14 - INFO - resample: False - scale factor 1 - true pixel 30.0
2022/02/09 12:08:14 - INFO - no bbox - full match: True - area: 1
2022/02/09 12:08:14 - INFO - calibrating band
2022/02/09 12

Looks pretty nice, right?

Now, let's create a custom logger that prints the data to the stderr and to a file instead.

There are two ways of customize the logging. 
- One is at the library level. That is, we change the default logger that is used in across the library.
- The other alternative is creating a logger that is passed to the different functions and classes (`logger` keyword).

Let's use the second approach in the next example.

## Custom logging instance

Let's first create a custom loggin instance.

In [4]:
import logging

logger = logging.getLogger("my_logger")

# Use a different log format
fmt = logging.Formatter(
    "[%(asctime)s] [%(levelname)s] %(message)s", datefmt="%Y/%m/%d %H:%M:%S"
)

## Add two handlers

# This one shows logs with a level equal or higher than INFO
stderr_handler = logging.StreamHandler()
stderr_handler.setLevel(logging.INFO)
stderr_handler.setFormatter(fmt)
logger.addHandler(stderr_handler)

# This one shows logs with a level equal or higher than DEBUG
import os

if os.path.isfile("important_logs.log"):
    os.unlink("important_logs.log")
filedump_handler = logging.FileHandler("important_logs.log")
filedump_handler.setLevel(logging.DEBUG)
filedump_handler.setFormatter(fmt)
logger.addHandler(filedump_handler)
logger.setLevel(logging.DEBUG)
logger

<Logger my_logger (DEBUG)>

Now, let's use it in the same task as before.

In [5]:
# With custom logger!
lp = LandsatLoader(IMAGES_RAW, bands=LANDSAT5_BANDS, spacecraft=5, logger=logger)

product_id = "LT05_L1TP_230094_20111011_20161005_01_T1"
res_ = lp.execute(
    product_id,
    bbox=None,
    folder_proc_=PROD_INT_DIR,
    raster_return_open=False,
    write_file="_complete",
)

[2022/02/09 12:09:40] [INFO] Running on Landsat 5 data
[2022/02/09 12:09:40] [INFO] Processing on ./products-int/images-raw/LT05_L1TP_230094_20111011_20161005_01_T1 dir
[2022/02/09 12:09:40] [INFO] Using clipping flag: False
[2022/02/09 12:09:40] [INFO] PROCESSING band: B1
[2022/02/09 12:09:40] [INFO] resample: False - scale factor 1 - true pixel 30.0
[2022/02/09 12:09:40] [INFO] no bbox - full match: True - area: 1
[2022/02/09 12:09:41] [INFO] calibrating band
[2022/02/09 12:09:43] [INFO] reprojecting band
[2022/02/09 12:09:52] [INFO] PROCESSING band: B2
[2022/02/09 12:09:52] [INFO] resample: False - scale factor 1 - true pixel 30.0
[2022/02/09 12:09:52] [INFO] no bbox - full match: True - area: 1
[2022/02/09 12:09:52] [INFO] calibrating band
[2022/02/09 12:09:55] [INFO] reprojecting band
[2022/02/09 12:10:03] [INFO] PROCESSING band: B3
[2022/02/09 12:10:03] [INFO] resample: False - scale factor 1 - true pixel 30.0
[2022/02/09 12:10:03] [INFO] no bbox - full match: True - area: 1
[202

Let's see the content of the log file.

In [6]:
with open("important_logs.log") as f:
    print(f.read())

[2022/02/09 12:09:40] [INFO] Running on Landsat 5 data
[2022/02/09 12:09:40] [INFO] Processing on ./products-int/images-raw/LT05_L1TP_230094_20111011_20161005_01_T1 dir
[2022/02/09 12:09:40] [INFO] Using clipping flag: False
[2022/02/09 12:09:40] [INFO] PROCESSING band: B1
[2022/02/09 12:09:40] [INFO] resample: False - scale factor 1 - true pixel 30.0
[2022/02/09 12:09:40] [INFO] no bbox - full match: True - area: 1
[2022/02/09 12:09:41] [INFO] calibrating band
[2022/02/09 12:09:43] [INFO] reprojecting band
[2022/02/09 12:09:52] [INFO] PROCESSING band: B2
[2022/02/09 12:09:52] [INFO] resample: False - scale factor 1 - true pixel 30.0
[2022/02/09 12:09:52] [INFO] no bbox - full match: True - area: 1
[2022/02/09 12:09:52] [INFO] calibrating band
[2022/02/09 12:09:55] [INFO] reprojecting band
[2022/02/09 12:10:03] [INFO] PROCESSING band: B3
[2022/02/09 12:10:03] [INFO] resample: False - scale factor 1 - true pixel 30.0
[2022/02/09 12:10:03] [INFO] no bbox - full match: True - area: 1
[202

We got the same output as the stderr handler, as expected since they were not debug messages.

## Set a new library-wide logger

Now, let's repeat the previous example but this time we update the default logger used by the eo-forge library.

In [7]:
logger = logging.getLogger("my_logger")
#
# IMPORTANT: We clean the logger to create a new one from scratch
# Otherwise, the handlers are added to the logger previously defined.
while logger.hasHandlers():
    logger.removeHandler(logger.handlers[0])

# Use a different log format than the other examples
fmt = logging.Formatter(
    " %(asctime)s || %(levelname)s || %(message)s", datefmt="%Y/%m/%d %H:%M:%S"
)

## Add two handlers

# This one shows logs with a level equal or higher than INFO
stderr_handler = logging.StreamHandler()
stderr_handler.setLevel(logging.INFO)
stderr_handler.setFormatter(fmt)
logger.addHandler(stderr_handler)

# This one shows logs with a level equal or higher than DEBUG
# Use a new file name.
if os.path.isfile("important_logs2.log"):
    os.unlink("important_logs2.log")
filedump_handler = logging.FileHandler("important_logs2.log")
filedump_handler.setLevel(logging.DEBUG)
filedump_handler.setFormatter(fmt)
logger.addHandler(filedump_handler)
logger.setLevel(logging.DEBUG)

####################################
# Set the new logger used by default
eo_forge.set_default_logger(logger)

In [8]:
# No custom logger passed this time as a keyword.
lp = LandsatLoader(IMAGES_RAW, bands=LANDSAT5_BANDS, spacecraft=5, logger=None)

product_id = "LT05_L1TP_230094_20111011_20161005_01_T1"
res_ = lp.execute(
    product_id,
    bbox=None,
    folder_proc_=PROD_INT_DIR,
    raster_return_open=False,
    write_file="_complete",
)

 2022/02/09 12:11:32 || INFO || Running on Landsat 5 data
 2022/02/09 12:11:32 || INFO || Processing on ./products-int/images-raw/LT05_L1TP_230094_20111011_20161005_01_T1 dir
 2022/02/09 12:11:32 || INFO || Using clipping flag: False
 2022/02/09 12:11:32 || INFO || PROCESSING band: B1
 2022/02/09 12:11:32 || INFO || resample: False - scale factor 1 - true pixel 30.0
 2022/02/09 12:11:32 || INFO || no bbox - full match: True - area: 1
 2022/02/09 12:11:32 || INFO || calibrating band
 2022/02/09 12:11:35 || INFO || reprojecting band
 2022/02/09 12:11:44 || INFO || PROCESSING band: B2
 2022/02/09 12:11:44 || INFO || resample: False - scale factor 1 - true pixel 30.0
 2022/02/09 12:11:44 || INFO || no bbox - full match: True - area: 1
 2022/02/09 12:11:44 || INFO || calibrating band
 2022/02/09 12:11:46 || INFO || reprojecting band
 2022/02/09 12:11:56 || INFO || PROCESSING band: B3
 2022/02/09 12:11:56 || INFO || resample: False - scale factor 1 - true pixel 30.0
 2022/02/09 12:11:56 || I